## Distill Bert

In [36]:
#Importing Libraries
import tensorflow as tf
'''
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
assert num_gpus_available > 0
'''
!pip install transformers
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [93]:
df = pd.read_excel("train.xlsx")
df

,Polarity,Review
0,5,dr. goldberg offers everything i look for in a...
1,2,"Unfortunately, the frustration of being Dr. Go..."
2,4,Been going to Dr. Goldberg for over 10 years. ...
3,4,Got a letter in the mail last week that said D...
4,1,I don't know what Dr. Goldberg was like before...
...,...,...
1995,4,I love Olive or Twist. Absolutely love it. As ...
1996,1,I've actually had OK experiences at this place...
1997,4,Girls night had to happen. We decided to go to...
1998,1,Owner or manager was very rude when we present...


In [94]:
#redefining polarity
df["Polarity"] = df["Polarity"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Polarity'] = df['Polarity'].map({'positive':1, 'negative':0})
df = df[["Review", "Polarity"]]

In [95]:
reviews = df['Review'].values.tolist()
labels = df['Polarity'].tolist()

In [ ]:
reviews

In [96]:
#divinding test, training and validation data set
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [97]:
#Assign tokenizer object to the tokenizer class
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [98]:
tokenizer([training_sentences[0]], truncation=True, padding=True, max_length=128)

{'input_ids': [[101, 7224, 1998, 2326, 2001, 2307, 1012, 10439, 25090, 16750, 2020, 2204, 1012, 2005, 4596, 2057, 8823, 1996, 15960, 8616, 4135, 2378, 1998, 1037, 16806, 10893, 1997, 2070, 4066, 1012, 2119, 12595, 2204, 2007, 1996, 15960, 8616, 18964, 2108, 1996, 2190, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [99]:
train_encodings = tokenizer(training_sentences, truncation=True, padding=True)
val_encodings = tokenizer(validation_sentences, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_labels))
val_dataset = tf.data.Dataset.from_tensor_slices(( dict(val_encodings), validation_labels ))

In [100]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

In [102]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
100/100 [==============================] - 4756s 47s/step - loss: 0.1105 - accuracy: 0.9619 - val_loss: 0.4236 - val_accuracy: 0.8575
Epoch 2/2
100/100 [==============================] - 4744s 47s/step - loss: 0.0583 - accuracy: 0.9806 - val_loss: 0.6549 - val_accuracy: 0.8350


In [103]:
model.save_pretrained("./sentiment")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")

Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [137]:
test_sentence = "This is a really good product. I love it"
test_sentence1 = "This is a really bad product. I hate it"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

In [142]:
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
print("Sentence: ",test_sentence)
tf.print("\nThe Prediction value :",tf_prediction[0][1])
if (tf_prediction[0][1]>=0.5):
  print("\nThe Review is Positive")
else:
  print("\nThe Review is Negetive")

Sentence:  This is a really good product. I love it

The Prediction value : 0.00230982807

The Review is Negetive


In [143]:
print("Sentence: ",test_sentence1)
predict_input = tokenizer.encode(test_sentence1,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)

tf.print("\nThe Prediction value :",tf_prediction[0][1])
if (tf_prediction[0][1]>=0.5):
  print("\nThe Review is Positive")
else:
  print("\nThe Review is Negetive")

Sentence:  This is a really bad product. I hate it

The Prediction value : 0.00230982807

The Review is Negetive
